In [1]:
#Approach:
# Load ground states into a Class.
# Step through all states (levels) loading isospin
# Loop through stable states
#   Get excitations with M1, get M1, isospin, energy
#   Calculate T3 #TODO
# Loop through unstable states
#   Find decays that can produce stable states with excitations we care about, get logfts #TODO
# Step through M1s
#   Calculate isospin rotation coefficient #TODO
#   Calculate BGT0 #TODO
#   Store in structure #TODO
# Plot #TODO
import numpy as np
import utils
import matplotlib.pyplot as plt
import os
import math
import pandas as pd
import re

m_e = 0.51099895069 #MeV/c
m_H = 1.007825032 #MeV/c
m_n = 1.008664916 #MeV/c
amu_to_MeV = 931.49410242
mass_excess_H = 7.2890
mass_excess_n = 8.0713

eps=0.5 #keV, states don't always line up in ensdf files, we use this to determine if two states are close enough to be considered the same



In [2]:
#Code for parsing ground JP string, which can have weird formatting
def parseJP(JP):
  #Store original JP in case our isolation of the first value causes an issue determining parity
  original_JP = JP
  #Remove weird characters
  JP = re.sub(r'[()\[\]]|GE|:',' ', JP)
  #Sometimes multiple JP--we assume the first one
  JPParts = JP.split()
  JP = JPParts[0] 

  #First get parity, then remove
  #Sometimes have a weird format, ie 1,2+
  if "+" in JP:
    parity = 1
  elif "-" in JP:
    parity =  -1
  elif "+" in original_JP:
    parity = 1
  elif "-" in original_JP:
    parity = -1
  else:
    print(f"Original JP is {original_JP}, cannot determine parity. Not a fatal error")
    parity=1
  
  #Now get rid of parity to parse J
  JP = JP.replace("+", "").replace("-", "")

  #Deal with fractions
  if "/" in JP:
    num,den = JP.split("/")
    J = float(num)/float(den)
  else:
    J = float(JP)

  return J,parity

#Generates an array of valid (stable) ground states indexed by [Z][N]
def parseStableGroundStates(fname):
  groundState_dicts = []

  readFile_df = pd.read_csv(fname,dtype=str,keep_default_na=False)
  for _,row in readFile_df.iterrows():
    iso_dict = {}
    iso_dict["Z"] = int(row["z"])
    iso_dict["N"] = int(row["n"])
    iso_dict["symbol"] = row["symbol"]
    T_str = row["isospin"] #can be complex, we need to parse
    iso_dict["T3"] = float(iso_dict["Z"]-iso_dict["N"])/2.
    if row["half_life"]=="STABLE":
      iso_dict["type"] = "STABLE"
      JP_str = row["jp"]
      iso_dict["mass"] = float(row["atomic_mass"])*amu_to_MeV*math.pow(10,-6)
      iso_dict["mass_excess"] = float(row["massexcess"])/1000.
      iso_dict["mass_unc"] = float(row["unc_am"])
      #JP is very simple for stable isotopes
      iso_dict["J"],iso_dict["parity"] = parseJP(JP_str)
    else:
      iso_dict["type"]="UNSTABLE"
      iso_dict["J"], iso_dict["parity"], iso_dict["mass"], iso_dict["mass_unc"], iso_dict["mass_excess"] = None, None, None, None, None
      if not row["atomic_mass"] == "":
        iso_dict["mass"] = float(row["atomic_mass"])*amu_to_MeV*math.pow(10,-6)
      if not row["unc_am"] == "":
        iso_dict["mass_unc"] = float(row["unc_am"])
      if not row["massexcess"] == "":
        iso_dict["mass_excess"] = float(row["massexcess"])/1000.

    #Most gnd states do not have isospin defined
    #We use:
    #   0 if N==Z and an even-even nuclus
    #   None if N==Z and an odd-odd or even-odd nucleus
    #   |T3| if N!=Z
    #   ENSDF value if assigned
    if T_str=="": #Not assigned
      if iso_dict["N"]==iso_dict["Z"]:
        if iso_dict["N"]%2==0 and iso_dict["Z"]%2==0: #even even
          iso_dict["T"]=0
          iso_dict["T_status"] = "Assigned"
        else:
          iso_dict["T"] = None
          iso_dict["T_status"] = None
      else:
        iso_dict["T"] = abs(iso_dict["T3"])
        iso_dict["T_status"] = "Assigned"
    else:
      if "/" in T_str:
        num,den = T_str.split("/")
        iso_dict["T"] = float(num)/float(den)
      else:
        iso_dict["T"] = float(T_str)
      iso_dict["T_status"] = "ENSDF"
      
    groundState_dicts.append(iso_dict)

  df = pd.DataFrame(groundState_dicts)
  return df.set_index(['Z', 'N'], drop=False)


In [3]:
# 1. Get ground states
# Parses the output of the call: nds.iaea.org/relnsd/v1/data?fields=ground_states&nuclides=all 
# to load properties of the ground states of all nuclei.
# Specifically we populat the GroundState class with
# Z, N, Symbol, 2*spin, parity, 2*isospin, mass, mass_unc
# and then have placeholder lists of the excited states and decays
df_groundStates = parseStableGroundStates("data/all.csv")

stable_with_isospin = df_groundStates[(df_groundStates["type"] == "STABLE") & (df_groundStates["T"].notna())]
unstable_with_isospin = df_groundStates[(df_groundStates["type"] == "UNSTABLE") & (df_groundStates["T"].notna())]
stable_with_isospin

Z    N symbol    T3    type    J  parity           mass  mass_unc  \
Z  N                                                                         
1  0     1    0      H   0.5  STABLE  0.5     1.0     938.783073  0.000014   
2  1     2    1     He   0.5  STABLE  0.5     1.0    2809.413526  0.000060   
   2     2    2     He   0.0  STABLE  0.0     1.0    3728.401326  0.000160   
3  3     3    3     Li   0.0  STABLE  1.0     1.0    5603.051495  0.001550   
   4     3    4     Li  -0.5  STABLE  1.5    -1.0    6535.365822  0.004500   
...     ..  ...    ...   ...     ...  ...     ...            ...       ...   
81 122  81  122     Tl -20.5  STABLE  0.5     1.0  189067.541482  1.257000   
   124  81  124     Tl -21.5  STABLE  0.5     1.0  190932.470194  1.330000   
82 124  82  124     Pb -21.0  STABLE  0.0     1.0  191863.999592  1.228000   
   125  82  125     Pb -21.5  STABLE  0.5    -1.0  192796.827232  1.231000   
   126  82  126     Pb -22.0  STABLE  0.0     1.0  193729.024784  1.232000   

        mass_excess     T  T_status  
Z  N                                 
1  0       7.288971   0.5  Assigned  
2  1      14.931219   0.5  Assigned  
   2       2.424916   0.0     ENSDF  
3  3      14.086880   0.0     ENSDF  
   4      14.907105   0.5     ENSDF  
...             ...   ...       ...  
81 122   -25.761309  20.5  Assigned  
   124   -23.820802  21.5  Assigned  
82 124   -23.785506  21.0  Assigned  
   125   -22.451968  21.5  Assigned  
   126   -21.748519  22.0  Assigned  

[243 rows x 12 columns]

In [4]:
#Code for parsing gammas and levels files - stored in a dict indexed by excited level #    
def parseGammas(symbol):
  levels = {}

  #Get level energy, twoJ, parity, m1, m1_unc
  readFile_df = pd.read_csv(f"data/gammas/{symbol}.csv",dtype=str,keep_default_na=False)
  if "end_level_idx" not in readFile_df.columns or "start_level_idx" not in readFile_df.columns:
    print(symbol)
    return {}
  
  df_filtered = readFile_df[(readFile_df["end_level_idx"] == "0") & (readFile_df["b_m1"] != "")]
  for _,row in df_filtered.iterrows():
    level_dict = {}
    level_dict["Z"] = int(row["z"])
    level_dict["N"] = int(row["n"])
    level_dict["lvl_idx"] = int(row["start_level_idx"])
    level_dict["energy"] = float(row["start_level_energy"])
    level_dict["BM1"] = float(row["b_m1"])

    if not row["unc_bm1"]=="":
      level_dict["BM1_unc"] = float(row["unc_bm1"])
    else:
      level_dict["BM1_unc"] = 0

    #JP is very simple for these well-defined levels, not generally true in ENSDF
    JP_str = row["start_level_jp"]
    level_dict["J"],level_dict["parity"] = parseJP(JP_str)

    #Placeholde,r we'll overwrite in next step if level has a known isopin
    level_dict["T"] = None
    level_dict["T_status"] = None

    levels[level_dict["lvl_idx"]]=level_dict
   
  #Get isospin
  readFile_df = pd.read_csv(f"data/levels/{symbol}.csv",dtype=str,keep_default_na=False)
  for _,row in readFile_df.iterrows():
    T_str = row["isospin"]  
    #If we have valid isospin, get level index
    lvl_idx = int(row["idx"])

    if lvl_idx in levels:
      #We want to restrict to well-defined isospins only
      if T_str == " " or T_str=="":
        continue #We can try to assign

      if "&" in T_str or "+" in T_str or " " in T_str:
        levels[lvl_idx]["T_status"] = "ENSDF"
        continue #Don't try to assign as ENSDF is unsure

      #Parse isospin
      if "/" in T_str:
        num,den = T_str.split("/")
        levels[lvl_idx]["T"] = float(num)/float(den)
        levels[lvl_idx]["T_status"] = "ENSDF"
      else:
        levels[lvl_idx]["T"] = float(T_str)
        levels[lvl_idx]["T_status"] = "ENSDF"
              
  return levels

In [5]:
# 2. Get M1 levels, isospins
#Parses file generated from  nds.iaea.org/relnsd/v1/data?fields=gammas&nuclides=<A><symbol>
#to generate a list of the excited states of a specifiic nucleus. The main point here
#is to get the energy, twoJ, parity, and B(M1) value.
#
#Then parses output of nds.iaea.org/relnsd/v1/data?fields=levels&nuclides=<A><symbol>
#to get the isospin of thes excited states.
all_levels = []
for _,entry in df_groundStates.iterrows():
  if entry["type"]=="STABLE":
    A = int(entry["Z"] + entry["N"])
    symbol = str(A)+entry["symbol"]
    levels = parseGammas(symbol)
    all_levels.extend(list(levels.values()))
    

#Now try to assign isopin of excited states to the isospin of the gnd state, up to the IAS
for level in all_levels:
  if level["T_status"] is None:
    deltaEc = 1.44*(level["Z"]+0.5)*math.pow(level["Z"]+level["N"],-1./3.) - 1.13
    if df_groundStates.loc[(level["Z"]+1,level["N"]-1)]["mass_excess"] is not None:
      Ex_ias = deltaEc + df_groundStates.loc[(level["Z"]+1,level["N"]-1)]["mass_excess"] - df_groundStates.loc[(level["Z"],level["N"])]["mass_excess"] + mass_excess_H - mass_excess_n
      if level["energy"]/1000. < Ex_ias-0.5:
        level["T"] = df_groundStates.loc[(level["Z"],level["N"])]["T"]
        level["T_status"] = "Assigned"
    
df_levels = pd.DataFrame(all_levels)
df_levels_with_isospin = df_levels[df_levels["T"].notna()]

1H
2H
3He
4He
Original JP is (1), cannot determine parity. Not a fatal error


In [6]:
#Code for parsing bm and bp files containing information about beta decays
#Pass in either beta+ or beta- folder, get list of files, step through parsing and extracting relevant info.
#We'll store this either in bms or bps
def parseDecays(folderName):
  fnames = [folderName+i for i in os.listdir(folderName) if i.endswith(".csv")]
  decays = []
  for fname in fnames:
    readFile_df = pd.read_csv(fname,dtype=str,keep_default_na=False)
    for _,row in readFile_df.iterrows():
      log_ft=row["log_ft"]
      hl_sec=row["half_life_sec"]
      if log_ft=="" or hl_sec=="":
        continue
      decay_dict = {"log_ft" : float(log_ft),
                    "half_life_sec" : float(hl_sec),
                    "parent_Z" : int(row["p_z"]),
                    "parent_N" : int(row["p_n"]),
                    "d_Z" : int(row["d_z"]),
                    "d_N" : int(row["d_n"]),
                    }
      
      JP_str = row["jp"]
      #JP parsing
      if not JP_str=="":
        decay_dict["J"],decay_dict["parity"] = parseJP(JP_str)
      else:
        decay_dict["J"],decay_dict["parity"] = None,None

      daughter_energy_str = row["daughter_level_energy"]
      if "+" in daughter_energy_str or "X" in daughter_energy_str:
        decay_dict["daughter_energy"] = None
      else:
        decay_dict["daughter_energy"] = float(daughter_energy_str)/1000.
        
      half_life_sec_unc_str = row["unc_hls"]
      if half_life_sec_unc_str=="":
        decay_dict["half_life_sec_unc"] = 0
      else:
        decay_dict["half_life_sec_unc"] = float(half_life_sec_unc_str)

      log_ft_unc_str = row["unc_lf"]
      if log_ft_unc_str=="":
        decay_dict["log_ft_unc"] = 0
      else:
        decay_dict["log_ft_unc"] = float(log_ft_unc_str)
        
      if "bm" in folderName:
        decay_dict["type"] = "bm"
      else:
        decay_dict["type"] = "bp"
      
      decays.append(decay_dict)
  return decays

In [13]:

#3. Step through decays. We will populate the levels objects in the gnd states with
#bm_decay and/or bp_decay Decay objects containing info about the decay to that specific level
bm_decays = parseDecays("data/bms/")
bp_decays = parseDecays("data/bps/")

all_decay_list = bm_decays + bp_decays
df_decays = pd.DataFrame(all_decay_list)
df_decays.sort_values(by=['parent_Z', 'parent_N'], inplace=True)
df_decays

Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), cannot determine parity. Not a fatal error
Original JP is (1 2), can

,log_ft,half_life_sec,parent_Z,parent_N,d_Z,d_N,J,parity,daughter_energy,half_life_sec_unc,log_ft_unc,type
4897,3.0170,6.139000e+02,0,1,1,0,0.5,1.0,0.00000,0.600000,0.0005,bm
5928,3.0524,3.887813e+08,1,2,2,1,0.5,1.0,0.00000,631138.519492,0.0008,bm
419,2.9059,8.067000e-01,2,4,3,3,0.0,1.0,0.00000,0.001500,0.0007,bm
10267,2.9100,1.191000e-01,2,6,3,5,0.0,1.0,9.67000,0.001200,0.0000,bm
10268,4.5300,1.191000e-01,2,6,3,5,0.0,1.0,5.15000,0.001200,0.0000,bm
...,...,...,...,...,...,...,...,...,...,...,...,...
12320,6.6000,4.662000e+03,101,155,100,156,1.0,-1.0,1.36040,108.000000,0.0000,bp
12321,6.2000,4.662000e+03,101,155,100,156,1.0,-1.0,1.32613,108.000000,0.0000,bp
12322,6.4000,4.662000e+03,101,155,100,156,1.0,-1.0,1.40522,108.000000,0.0000,bp
12323,6.6000,4.662000e+03,101,155,100,156,1.0,-1.0,0.00000,108.000000,0.0000,bp


In [8]:
def safeFactorial(val):
  return math.factorial(int(val))

# Algorithmic calculation of CG coefficients, based on 
# https://en.wikipedia.org/wiki/Clebsch%E2%80%93Gordan_coefficients#Relation_to_Wigner_3-j_symbols
def getClebschGordonCoefficient(j1,j2,m1,m2,J,M):
  #Delta fn
  if not M==m1+m2:
    return 0
  
  #term 1
  numerator = (2*J+1)*safeFactorial(j1+j2-J)*safeFactorial(j1-j2+J)*safeFactorial(-j1+j2+J)
  denominator = safeFactorial(j1+j2+J+1)
  if denominator==0:
    return 0
  else:
    term1 = math.sqrt(numerator/denominator)
    
  #term 2
  numerator=safeFactorial(j1-m1)*safeFactorial(j1+m1)*safeFactorial(j2-m2)*safeFactorial(j2+m2)*safeFactorial(J-M)*safeFactorial(J+M)
  term2 = math.sqrt(numerator)

  #term 3
  k_min = max([0,j2-J-m1,j1-J+m2])
  k_max = min([j1+j2-J,j1-m1,j2+m2])
  term3=0
  k = k_min
  while k<=k_max:
    numerator=math.pow(-1,k)

    f1 = safeFactorial(k)
    f2 = safeFactorial(j1+j2-J-k)
    f3 = safeFactorial(j1-m1-k)
    f4 = safeFactorial(j2+m2-k)
    f5 = safeFactorial(J-j2+m1+k)
    f6 = safeFactorial(J-j1-m2+k)
    
    denominator = f1*f2*f3*f4*f5*f6
    if denominator==0:
      return 0
    else:
      term3 += numerator/denominator
    
    k=k+1
    
  CG=term1*term2*term3
  print(f"CG is {CG}")
  return CG


In [ ]:
# 4. Step through M1 transitions, calculating rotation coefficent, BGT0. Add to level
#Requirements:
#   Ground state is stable
#   Ground state has an isospin
#   Levels has an isospin
#   Leve1s has a BM1
#   Parent decay has an isospin
#   Parent decay has a half-life
#   Parent decay has a logft

for i,j in np.ndindex(ground_states.shape):
  ground_state = ground_states[i][j]
  if ground_state is None or ground_state.atomType=="UNSTABLE":
    continue

  for key in ground_state.levels.keys():
    #Check we have a level and gnd state isospin defined, and a bm1 value
    if ground_state.levels[key].T is None or ground_state.T is None or ground_state.levels[key].bm1 is None:
      continue

    bm_ga_BGT0_squared = None
    bp_ga_BGT0_squared = None

    #Check if we have a bm decay
    if ground_state.levels[key].bm_decay is not None:

      #Check that bm decay has a half life, logft
      if ground_state.levels[key].bm_decay.logft is None or ground_state.levels[key].bm_decay.half_life_sec is None:
        continue

      #Check the bm_parent has a T
      if ground_state.levels[key].bm_decay.parent_T is None:
        continue
      
      #Check the beta decay parent isospin matches the ground state's
      if not ground_state.T == ground_state.levels[key].bm_decay.parent_T:
        continue

      #Calculate CG coefficient in the numerator -  NC term
      j1 = ground_state.T
      m1 = ground_state.T3
      j2 = 1
      m2 = 0
      J = ground_state.levels[key].T
      M = m1 #Should be same as initial

      numerator = getClebschGordonCoefficient(j1,j2,m1,m2,J,M)

      #Calculate CG coefficient in the denominator
      j1 = ground_state.levels[key].bm_decay.parent_T
      m1 = ground_state.levels[key].bm_decay.parent_T3
      j2 = 1
      m2 = -1 #-1 for beta-
      J = ground_state.levels[key].T
      M = ground_state.levels[key].bm_decay.parent_T + 1 #+1 for beta+

      denominator = getClebschGordonCoefficient(j1,j2,m1,m2,J,M)

      #Calculate (sqrt(2) * num/den)^2
      t1 = 2 * math.pow(numerator/denominator,2)

      #Calculate half life term
      t2 = 6165/math.pow(10,ground_state.levels[key].bm_decay.logft)

      #Calculate BGT0, store in level
      bm_ga_BGT0_squared = t1*t2
    

    #Check if we have a bp decay
    if ground_state.levels[key].bp_decay is not None:

      #Check that bm decay has a half life, logft
      if ground_state.levels[key].bp_decay.logft is None or ground_state.levels[key].bp_decay.half_life_sec is None:
        continue

      #Check the bm_parent has a T
      if ground_state.levels[key].bp_decay.parent_T is None:
        continue
      
      #Check the beta decay parent isospin matches the ground state's
      if not ground_state.T == ground_state.levels[key].bp_decay.parent_T:
        continue

      #Calculate CG coefficient in the numerator -  NC term
      j1 = ground_state.T
      m1 = ground_state.T3
      j2 = 1
      m2 = 0
      J = ground_state.levels[key].T
      M = m1 #Should be same as initial

      numerator = getClebschGordonCoefficient(j1,j2,m1,m2,J,M)

      #Calculate CG coefficient in the denominator
      j1 = ground_state.levels[key].bp_decay.parent_T
      m1 = ground_state.levels[key].bp_decay.parent_T3
      j2 = 1
      m2 = 1 #1 for beta+/EC
      J = ground_state.levels[key].T
      M = ground_state.levels[key].bp_decay.parent_T - 1 #-1 for b+/EC

      denominator = getClebschGordonCoefficient(j1,j2,m1,m2,J,M)
      if denominator==0:
        continue

      #Calculate (sqrt(2) * num/den)^2
      t1 = 2 * math.pow(numerator/denominator,2)

      #Calculate half life term
      t2 = 6165/math.pow(10,ground_state.levels[key].bp_decay.logft)

      #Calculate BGT0, store in level
      bp_ga_BGT0_squared = t1*t2

    #Average if needed, store in levels.ga_BGT0_squared
    if bm_ga_BGT0_squared is None and bp_ga_BGT0_squared is not None:
      ground_states[i][j].levels[key].ga_BGT0_squared = bp_ga_BGT0_squared
    elif bm_ga_BGT0_squared is not None and bp_ga_BGT0_squared is None:
      ground_states[i][j].levels[key].ga_BGT0_squared = bm_ga_BGT0_squared
    elif bm_ga_BGT0_squared is not None and bp_ga_BGT0_squared is not None:
      ground_states[i][j].levels[key].ga_BGT0_squared = (bm_ga_BGT0_squared + bp_ga_BGT0_squared)/2.

#Step 5. Plot BGT0 vs BM1
bm1s=[]
bgt0s=[]
for i,j in np.ndindex(ground_states.shape):
  ground_state = ground_states[i][j]
  if ground_state is None:
    continue

  for key in ground_state.levels.keys():
    if ground_states[i][j].levels[key].ga_BGT0_squared is not None and ground_states[i][j].levels[key].bm1 is not None:
      bm1s.append(ground_states[i][j].levels[key].bm1)
      bgt0s.append(ground_states[i][j].levels[key].ga_BGT0_squared)

plt.scatter(bm1s,bgt0s)
plt.show()

NameError: name 'ground_states' is not defined